In [1]:
from gensim.models import Word2Vec
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df = pd.read_csv('questions_KU.csv')
df.dropna(inplace=True)
df.head()

,Unnamed: 0,Filenumber,Question,Topic
0,0,694,एडस बीमारी क्या है ?,STDs
1,1,1112,सेक्स की बात करने से लिंग खड़ा क्यों हो जाता है,Other sexual health issues
2,2,5374,हस्तमैथुन करने के बाद लिंग में जलन होता है,Masturbation
3,3,3705,महिलाओं के योनि और स्तन के बारे में बताएँ,Other sexual health issues
4,4,3063,सेक्स के दौरान पत्नी चिल्लाती क्यों है,\N


In [3]:
def cleaning(data):
    for i in range(0, len(data)):
        temp = ""
        for j in range(0, len(data[i])):
            if(data[i][j] != ',' and data[i][j] != ')' and data[i][j] != '(' and data[i][j] != '"' and data[i][j] != ':' and data[i][j] != '%' and not(data[i][j] >= '0' and data[i][j] <= '9') and data[i][j] != '-' and data[i][j] != '/' and not(data[i][j] >= 'a' and data[i][j] <= 'z') and not(data[i][j] >= 'A'  and data[i][j] <= 'Z') and data[i][j] != '/' and data[i][j] != '-'):
                temp=temp+data[i][j]
        data[i]=temp
    return data

In [4]:
def stopwords_removal(data):
    stop_file=open("stopwords-hi.txt",encoding='UTF-8').read()
    stopwords=stop_file.split()
    dictionary={}
    for word in stopwords:
        dictionary[word]=1
    for i in  range(len(data)):
        temp = ""
        words = data[i].split(" ")
        for j in words:
            if j == '?':
                continue
            elif j not in dictionary:
                temp += (j + " ")
        data[i]=temp
    return data

In [5]:
def tokenization(data):
    corpus = []
    for questions in data:
        corpus.append(questions.split())
    return corpus

In [6]:
data_ques = df['Question']
data = data_ques.to_list()
print(data[130])
clean_data = cleaning(data)
preprocessed_data = stopwords_removal(clean_data)
print(preprocessed_data[130])
corpus = tokenization(preprocessed_data)
print(len(corpus))

लिंग डालने के लिए कितनी ताकत की जरुरत होती है
लिंग डालने कितनी ताकत जरुरत 
6658


## 1) Word2Vec trained on Questions only

In [7]:
model = Word2Vec(corpus, size = 300, window = 5, min_count = 2, negative = 5)

## 2) Word2Vec trained on Answers and Questions

In [17]:
ans = pd.read_csv('answers_KU.csv')
ans_np = ans['Answer']
ans_list = ans_np.to_list()
clean_ans = cleaning(ans_list)
preprocessed_ans = stopwords_removal(clean_ans)
corpus += tokenization(preprocessed_ans)
model = Word2Vec(corpus, size = 300, window = 5, min_count = 2, negative = 5)

## 3) Word2Vec trained on IITB Corpus

In [29]:
with open('monolingual.hi') as f:
    head = [next(f) for x in range(10000)]

In [30]:
sen = []
for line in head:
    words = line.split()
    sen.append(words)

In [31]:
for l in sen:
    corpus += l

## Training the model (Same for all)

In [32]:
def vectors(data):
    global word_embeddings
    word_embeddings = []

    for question in data:
        avgword2vec = None
        count = 0
        for word in question.split():
            if word in model.wv.vocab:
                count += 1
                if avgword2vec is None:
                    avgword2vec = model[word]
                else:
                    avgword2vec = avgword2vec + model[word]
                
        if avgword2vec is not None:
            avgword2vec = avgword2vec / count
        
            word_embeddings.append(avgword2vec)

In [33]:
vectors(preprocessed_data)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [34]:
model.wv.most_similar(positive=["एडस"])

[('क्षमता', 0.9922375082969666),
 ('मीडिया', 0.9921690225601196),
 ('बनाया', 0.9921455979347229),
 ('देखे', 0.9921245574951172),
 ('गर्भ', 0.9921217560768127),
 ('जिसकी', 0.9921210408210754),
 ('सफाई', 0.9921186566352844),
 ('आकर्षित', 0.9921162128448486),
 ('दोस्ती', 0.9921047687530518),
 ('तरीका', 0.9921005964279175)]

In [35]:
def recommendations(question):
    cosine_similarities = cosine_similarity(word_embeddings, word_embeddings)
    ques = df.loc[df['Question'] == question].drop_duplicates()
    index_val = ques.index
    sim_scores = list(enumerate(cosine_similarities[index_val[0]]))
    #print(sim_scores)
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
    sim_scores = sim_scores[1:6]
    print(sim_scores)
    ques_indices = [i[0] for i in sim_scores]
    recommend = df.iloc[ques_indices]
    for index, row in recommend.iterrows():
        print(row['Question'])

In [36]:
recommendations("प्रेमी ठीक से बात नही करता")

[(768, 0.9999996), (2646, 0.9999996), (3470, 0.99996984), (5339, 0.9999669), (304, 0.9999668)]
प्रेमी ठीक से बात नही करता
लिंग खड़ा नहीं होता है
सेक्स के दौरान क्यों लिंग योनि से चिपकने लगता है
गुद्दा मैथुन कैसे किया जाता है ?
सेक्स कैसे किया जाता है ?


In [37]:
recommendations("गर्ल-फ्रेंड बनाने के उपाय बताइये")

[(1893, 1.0000002), (2904, 1.0000002), (1488, 0.99996406), (4631, 0.99996406), (4877, 0.99996406)]
सेक्स करने में कैसा लगता है
कॉन्डम लगाकर नाबालिग के साथ सेक्स करना क्या सही है
किसी लड़की से बात करने पर लिंग खड़ा हो जाता है
लिंग मोटा और बड़ा कैसे हो सकता है
ज्यादा हस्थमैथुन करने से क्या लिंग कमजोर हो जाता है
